In [ ]:
from ratdata import data_manager as dm, process, plot as rdplot
import matplotlib.pyplot as plt
import numpy as np
import pickle

In [ ]:
dm.db_connect('rat_data.db')

In [ ]:
rec = dm.RecordingFile.select().where(dm.RecordingFile.filename=='2021-05-24T09-48-46 rat3 - OFT2.mat').get()
# rec = dm.RecordingFile.select().where(dm.RecordingFile.filename=='2021-07-16T09-02-07 rat3 OFT2 random.mat').get()
# rec = dm.RecordingFile.select().where(dm.RecordingFile.filename=='2021-07-12T09-23-24 rat3 OFT2 on-off.mat').get()
# rec = dm.RecordingFile.select().where(dm.RecordingFile.filename=='2021-05-19T08-57-35 rat3 - OFT2 130Hz.mat').get()

In [ ]:
data, dt, time = dm.get_electrode_data_from_recording(rec, select_slice=True)
mean_data = np.mean(data, axis=0)
fs = int(1/dt)
title = 'Biomarker for %s' % rec.filename

In [ ]:
# rdplot.plot_biomarker_steps(mean_data, fs, time=time, plot_title=title)

In [ ]:
perc = 50
s, env1, env2 = process.beta_envelope(mean_data, fs, 300)
th1 = np.percentile(env1, perc)
th2 = np.percentile(env2, perc)
bursts1 = process.beta_bursts(env1, th1)
bursts2 = process.beta_bursts(env2, th2)
print(th1, th2)
durations1 = [(e[1] - e[0]) / 300 for e in bursts1]
durations2 = [(e[1] - e[0]) / 300 for e in bursts2]
amplitudes1 = [e[2] for e in bursts1]
amplitudes2 = [e[2] for e in bursts2]
plt.figure(figsize=(20, 9))
ax1 = plt.subplot2grid((2, 2), (0, 0), colspan=2)
ax1.plot(s)
ax1.plot(env1)
ax1.plot(env2)
ax2 = plt.subplot2grid((2, 2), (1, 0))
ax2.scatter(durations1, amplitudes1)
ax3 = plt.subplot2grid((2, 2), (1, 1))
ax3.scatter(durations1, amplitudes1)

In [ ]:
perc = 50
for rat in dm.Rat.select():
    print(f'{rat.full_label}:')
    rat_bursts = dict()
    rat_bursts['perc'] = perc
    rat_bursts['rat'] = rat.full_label
    for s in dm.StimSettings.select(dm.StimSettings.stim_type).distinct():
        st = s.stim_type
        durations = []
        amplitudes = []
        print(st, ':')
        for recording in dm.RecordingFile.select().join(dm.StimSettings).where((dm.RecordingFile.rat == rat) & (dm.StimSettings.stim_type == st)):
            data, dt, time = dm.get_electrode_data_from_recording(recording, select_slice=True)
            mean_data = np.mean(data, axis=0)
            fs = int(1/dt)
            s, env1, _ = process.beta_envelope(mean_data, fs, 300)
            th1 = np.percentile(env1, perc)
            bursts1 = process.beta_bursts(env1, th1)
            durations1 = [(e[1] - e[0]) / 300 for e in bursts1]
            amplitudes1 = [e[2] for e in bursts1]
            amplitudes.extend(amplitudes1)
            durations.extend(durations1)
            stim_type = recording.stim.get_or_none().stim_type
            print(recording.filename, stim_type)
        rat_bursts[st] = {
            'amplitudes': amplitudes,
            'durations': durations
        }
    with open(f'data/beta_bursts_{perc}_{rat.full_label}.pickle', 'wb') as f:
        pickle.dump(rat_bursts, f)

In [ ]:
for perc in [50, 75]:
    for rat in dm.Rat.select():
        with open(f'data/beta_bursts_{perc}_{rat.full_label}.pickle', 'rb') as f:
            rat_bursts = pickle.load(f)
        perc = rat_bursts['perc']
        rat_label = rat_bursts['rat']
        rat_type = rat.group
        rdplot.plot_beta_bursts_one_rat(rat_bursts, rat_type, f'plots/beta_bursts/{perc}_{rat_label}')